# Assignment 8: Data Aggregation and Group Operations

## Overview
This assignment covers data aggregation and group operations using healthcare/insurance EHR data. You'll analyze patient encounters (insurance claims), healthcare providers, and medical facilities across different regions. The data includes provider statistics, facility information, and encounter charges with insurance coverage details.

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set random seed for reproducibility
np.random.seed(42)

# Create output directory
os.makedirs('output', exist_ok=True)

## Question 1: Basic GroupBy Operations

### Part 1.1: Load and Explore Data

**TODO: Load and merge the datasets**

In [36]:
# TODO: Load the three CSV files into dataframes
# provider_df = pd.read_csv(...)
# facility_df = pd.read_csv(...)
# encounter_df = pd.read_csv(...)
provider_df = pd.read_csv('data/provider_data.csv')
facility_df = pd.read_csv('data/facility_data.csv')
encounter_df = pd.read_csv('data/encounter_data.csv')

# TODO: Print the shapes of each dataframe
# print("Provider data shape:", ...)
# print("Facility data shape:", ...)
# print("Encounter data shape:", ...)
print("Provider data shape:", provider_df.shape)
print("Facility data shape:", facility_df.shape)
print("Encounter data shape:", encounter_df.shape)

# TODO: Merge the dataframes for analysis
# Hint: Start by merging encounter_df with provider_df on 'provider_id'
#       Then merge the result with facility_df on 'facility_id'
# Note: Both encounter_df and provider_df have 'facility_id', and both encounter_df and facility_df have 'region'
#       After merging, you may need to handle duplicate column names (pandas will add suffixes like _x and _y)
#       You can drop one of the duplicate columns and rename if needed
merged_df = pd.merge(encounter_df, provider_df, on='provider_id')
merged_df = merged_df.drop(columns=['facility_id_y'])
merged_df = merged_df.rename(columns={'facility_id_x': 'facility_id'})
merged_df = pd.merge(merged_df, facility_df, on='facility_id')
merged_df = merged_df.drop(columns=['region_y'])
merged_df = merged_df.rename(columns={'region_x': 'region'})

# TODO: Print the merged dataframe shape, columns, and first few rows
# print("\nMerged data shape:", ...)
# print("\nColumns:", ...)
# print("\nFirst few rows:")
# print(...)
print("\nMerged data shape:", merged_df.shape)
print("\nColumns:", merged_df.columns.tolist())
print("\nFirst few rows:")
print(merged_df.head())

Provider data shape: (500, 7)
Facility data shape: (10, 6)
Encounter data shape: (5000, 12)

Merged data shape: (5000, 21)

Columns: ['encounter_id', 'patient_id', 'provider_id', 'facility_id', 'encounter_date', 'encounter_type', 'diagnosis_code', 'procedure_code', 'service_charge', 'insurance_paid', 'patient_paid', 'region', 'provider_name', 'provider_type', 'specialty', 'years_experience', 'license_number', 'facility_name', 'facility_type', 'beds', 'established_date']

First few rows:
  encounter_id patient_id provider_id facility_id encounter_date  \
0     ENC00001   PAT01564      PR0259      FAC004     2023-08-21   
1     ENC00002   PAT00296      PR0258      FAC001     2023-09-11   
2     ENC00003   PAT01381      PR0213      FAC001     2023-03-20   
3     ENC00004   PAT01131      PR0146      FAC004     2023-09-03   
4     ENC00005   PAT01388      PR0354      FAC003     2023-06-14   

         encounter_type diagnosis_code  procedure_code  service_charge  \
0          Office Visit  

### Part 1.2: Basic Aggregation

**TODO: Perform basic groupby operations**

In [37]:
# TODO: Group by facility (healthcare facility) and calculate basic stats
# TODO: Calculate mean, sum, count for years_experience by facility
# TODO: Calculate total encounter charges by facility
# TODO: Calculate total insurance payments by facility
# TODO: Find the facility with highest total encounter charges
grouped = merged_df.groupby('facility_name').agg({
    'provider_id': 'nunique',
    'years_experience': ['mean', 'sum', 'count'],
    'service_charge': 'sum',
    'insurance_paid': 'sum'
}).reset_index().sort_values(by=('service_charge', 'sum'), ascending=False)
display(grouped)
# TODO: Create a summary dataframe with one row per facility showing:
#       facility_name, total_providers, avg_years_experience, total_service_charge, total_insurance_paid
# TODO: Save the summary dataframe as 'output/q1_groupby_analysis.csv'
summary_df = grouped.copy()
summary_df.columns = ['facility_name', 'total_providers', 'avg_years_experience', 'total_years_experience', 'count_years_experience', 'total_encounter_charge', 'total_insurance_paid']
summary_df = summary_df[['facility_name', 'total_providers', 'avg_years_experience', 'total_encounter_charge', 'total_insurance_paid']]
summary_df.to_csv('output/q1_groupby_analysis.csv', index=False)

facility_name provider_id years_experience               \
                                   nunique             mean    sum count   
6        Pediatric Care Center         326        19.811676  10520   531   
9    Surgical Specialty Center         340        20.778182  11428   550   
2      Community Health Clinic         328        20.160784  10282   510   
5    Oncology Treatment Center         324        19.381139   9865   509   
7          Primary Care Clinic         316        20.102970  10152   505   
8      Regional Medical Center         309        20.209205   9660   478   
3     Emergency Medical Center         299        19.668687   9736   495   
4        Northside Urgent Care         305        20.137014   9847   489   
1        City General Hospital         312        19.563008   9625   492   
0  Cardiology Specialty Center         297        19.319728   8520   441   

  service_charge insurance_paid  
             sum            sum  
6      361811.04      291790.95  
9      349909.18      278503.38  
2      334992.21      269207.92  
5      334927.30      267183.95  
7      323315.71      260297.44  
8      316284.98      251750.43  
3      310735.25      248439.20  
4      297283.06      236839.71  
1      295508.13      236798.50  
0      276658.83      222139.04

In [38]:
display(summary_df)

,facility_name,total_providers,avg_years_experience,total_encounter_charge,total_insurance_paid
6,Pediatric Care Center,326,19.811676,361811.04,291790.95
9,Surgical Specialty Center,340,20.778182,349909.18,278503.38
2,Community Health Clinic,328,20.160784,334992.21,269207.92
5,Oncology Treatment Center,324,19.381139,334927.30,267183.95
7,Primary Care Clinic,316,20.102970,323315.71,260297.44
8,Regional Medical Center,309,20.209205,316284.98,251750.43
3,Emergency Medical Center,299,19.668687,310735.25,248439.20
4,Northside Urgent Care,305,20.137014,297283.06,236839.71
1,City General Hospital,312,19.563008,295508.13,236798.50
0,Cardiology Specialty Center,297,19.319728,276658.83,222139.04


### Part 1.3: Transform Operations

**TODO: Use transform operations to add group statistics**

In [39]:
# TODO: Add facility mean years_experience as new column
# TODO: Add facility standard deviation of years_experience
# TODO: Create normalized years_experience (z-score within facility)
# TODO: Add facility total encounter charges as new column
summary_df['facility_mean_years_experience'] = summary_df.groupby('facility_name')['avg_years_experience'].transform('mean')
summary_df['facility_std_years_experience'] = summary_df.groupby('facility_name')['avg_years_experience'].transform('std')
summary_df['normalized_years_experience'] = summary_df.groupby('facility_name')['avg_years_experience'].transform(lambda x: (x - x.mean()) / x.std())
summary_df['facility_total_encounter_charges'] = summary_df.groupby('facility_name')['total_encounter_charge'].transform('sum')

# TODO: Display the enhanced dataframe
# TODO: Save results as 'output/q1_aggregation_report.txt'
display(summary_df)
summary_df.to_csv('output/q1_aggregation_report.txt', index=False)

,facility_name,total_providers,avg_years_experience,total_encounter_charge,total_insurance_paid,facility_mean_years_experience,facility_std_years_experience,normalized_years_experience,facility_total_encounter_charges
6,Pediatric Care Center,326,19.811676,361811.04,291790.95,19.811676,NaN,NaN,361811.04
9,Surgical Specialty Center,340,20.778182,349909.18,278503.38,20.778182,NaN,NaN,349909.18
2,Community Health Clinic,328,20.160784,334992.21,269207.92,20.160784,NaN,NaN,334992.21
5,Oncology Treatment Center,324,19.381139,334927.30,267183.95,19.381139,NaN,NaN,334927.30
7,Primary Care Clinic,316,20.102970,323315.71,260297.44,20.102970,NaN,NaN,323315.71
8,Regional Medical Center,309,20.209205,316284.98,251750.43,20.209205,NaN,NaN,316284.98
3,Emergency Medical Center,299,19.668687,310735.25,248439.20,19.668687,NaN,NaN,310735.25
4,Northside Urgent Care,305,20.137014,297283.06,236839.71,20.137014,NaN,NaN,297283.06
1,City General Hospital,312,19.563008,295508.13,236798.50,19.563008,NaN,NaN,295508.13
0,Cardiology Specialty Center,297,19.319728,276658.83,222139.04,19.319728,NaN,NaN,276658.83


## Question 2: Advanced GroupBy Operations

### Part 2.1: Filter Operations

**TODO: Use filter operations to remove groups**

In [40]:
# TODO: Filter facilities with more than 20 providers (sufficient staffing)
# TODO: Filter facilities with average years_experience > 10 (experienced staff)
# TODO: Filter facilities with total encounter charges > 250000 (high volume)
sufficient_staffing = merged_df.groupby('facility_name').filter(lambda x: x['provider_id'].nunique() > 20)
experienced_staff = merged_df.groupby('facility_name').filter(lambda x: x['years_experience'].mean() > 10)
high_volume = merged_df.groupby('facility_name').filter(lambda x: x['service_charge'].sum() > 250000)

# TODO: Create a summary of filtered results showing whether each facility meets the criteria
# TODO: Save results as 'output/q2_filter_analysis.csv'

### Part 2.2: Apply Operations

**TODO: Use apply operations with custom functions**

In [41]:
# TODO: Create custom function to calculate provider statistics
def provider_stats(group):
    # TODO: Return mean, std, min, max, range for years_experience
    stats = {
        'mean_years_experience': group['years_experience'].mean(),
        'std_years_experience': group['years_experience'].std(),
        'min_years_experience': group['years_experience'].min(),
        'max_years_experience': group['years_experience'].max(),
        'range_years_experience': group['years_experience'].max() - group['years_experience'].min()
    }
    return pd.Series(stats)

# TODO: Apply custom function to each facility
# Note: Add include_groups=False to avoid pandas FutureWarning about groupby columns
facility_provider_stats = merged_df.groupby('facility_name').apply(provider_stats, include_groups=False)

# TODO: Create function to find top providers in each facility
def top_providers(group, n=2):
    """Return top n providers in a group by encounter volume"""
    # TODO: Return top n rows by encounter count, selecting only provider_id, provider_name, 
    #       years_experience, specialty
    top_n = group.nlargest(n, 'encounter_id')
    return top_n[['provider_id', 'provider_name', 'years_experience', 'specialty']]

# TODO: Apply function to get top 2 providers per facility
# IMPORTANT: When using groupby().apply(), the result has a MultiIndex where:
# - Level 0: The grouping column (facility_name)
# - Level 1: The original row index from each group
# Use reset_index(level=0) to move only the grouping column back to a regular column
# DO NOT use reset_index(drop=True) - that would drop ALL index levels including facility_name!
top_providers_by_facility = merged_df.groupby('facility_name').apply(
    lambda x: top_providers(x, n=2)
).reset_index(level=0)  # Moves facility_name from index to column

# TODO: Save results as 'output/q2_performance_report.txt'
top_providers_by_facility.to_csv('output/q2_performance_report.txt', index=False)

TypeError: Column 'encounter_id' has dtype object, cannot use method 'nlargest' with this dtype

### Part 2.3: Hierarchical Grouping

**TODO: Perform multi-level grouping**

In [43]:
# TODO: Group by facility type and region
# TODO: Calculate statistics for each facility_type-region combination
# TODO: Use unstack to convert to wide format
# TODO: Use stack to convert back to long format
hierarchical_stats = merged_df.groupby(['facility_type', 'region']).agg({
    'years_experience': ['mean', 'std', 'min', 'max'],
    'service_charge': 'sum',
    'insurance_paid': 'sum'
})
hierarchical_stats_stacked = hierarchical_stats.stack()
print("hierarchical_stats_stacked")
display(hierarchical_stats_stacked)
hierarchical_stats_unstacked = hierarchical_stats_stacked.unstack()
print("hierarchical_stats_unstacked")
display(hierarchical_stats_unstacked)

# TODO: Analyze the hierarchical structure
# TODO: Save results as 'output/q2_hierarchical_analysis.csv'

hierarchical_stats_stacked


/var/folders/4l/xd_kd9gd62g2q0x2tmwl8z040000gn/T/ipykernel_22175/1172077652.py:10: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  hierarchical_stats_stacked = hierarchical_stats.stack()


years_experience  service_charge  insurance_paid
facility_type    region                                                       
Hospital         South  max          39.000000             NaN             NaN
                        mean         19.381139             NaN             NaN
                        min           1.000000             NaN             NaN
                        std          10.872386             NaN             NaN
                        sum                NaN       334927.30       267183.95
Specialty Center South  max          39.000000             NaN             NaN
                        mean         19.742690             NaN             NaN
                        min           1.000000             NaN             NaN
                        std          11.005988             NaN             NaN
                        sum                NaN       672546.29       540230.15
                 West   max          39.000000             NaN             NaN
                        mean         19.737844             NaN             NaN
                        min           1.000000             NaN             NaN
                        std          10.806376             NaN             NaN
                        sum                NaN       599974.54       482436.48
Urgent Care      North  max          39.000000             NaN             NaN
                        mean         20.160784             NaN             NaN
                        min           1.000000             NaN             NaN
                        std          11.044011             NaN             NaN
                        sum                NaN       334992.21       269207.92
                 South  max          39.000000             NaN             NaN
                        mean         19.881443             NaN             NaN
                        min           1.000000             NaN             NaN
                        std          11.140225             NaN             NaN
                        sum                NaN       611793.11       488548.93
                 West   max          39.000000             NaN             NaN
                        mean         20.476420             NaN             NaN
                        min           1.000000             NaN             NaN
                        std          10.998825             NaN             NaN
                        sum                NaN       647192.24       515343.09

hierarchical_stats_unstacked


years_experience                                 \
                                     max       mean  min        std sum   
facility_type    region                                                   
Hospital         South              39.0  19.381139  1.0  10.872386 NaN   
Specialty Center South              39.0  19.742690  1.0  11.005988 NaN   
                 West               39.0  19.737844  1.0  10.806376 NaN   
Urgent Care      North              39.0  20.160784  1.0  11.044011 NaN   
                 South              39.0  19.881443  1.0  11.140225 NaN   
                 West               39.0  20.476420  1.0  10.998825 NaN   

                        service_charge                         insurance_paid  \
                                   max mean min std        sum            max   
facility_type    region                                                         
Hospital         South             NaN  NaN NaN NaN  334927.30            NaN   
Specialty Center South             NaN  NaN NaN NaN  672546.29            NaN   
                 West              NaN  NaN NaN NaN  599974.54            NaN   
Urgent Care      North             NaN  NaN NaN NaN  334992.21            NaN   
                 South             NaN  NaN NaN NaN  611793.11            NaN   
                 West              NaN  NaN NaN NaN  647192.24            NaN   

                                                 
                        mean min std        sum  
facility_type    region                          
Hospital         South   NaN NaN NaN  267183.95  
Specialty Center South   NaN NaN NaN  540230.15  
                 West    NaN NaN NaN  482436.48  
Urgent Care      North   NaN NaN NaN  269207.92  
                 South   NaN NaN NaN  488548.93  
                 West    NaN NaN NaN  515343.09

## Question 3: Pivot Tables and Cross-Tabulations

### Part 3.1: Basic Pivot Tables

**TODO: Create pivot tables for multi-dimensional analysis**

In [ ]:
# TODO: Create pivot table: encounter charges by procedure code and region
# TODO: Create pivot table with multiple aggregations (sum, mean, count)
# TODO: Add totals (margins) to pivot table
# TODO: Handle missing values with fill_value
pivot_table = pd.pivot_table(
    merged_df,
    values='service_charge',
    index='procedure_code',
    columns='region',
    aggfunc=['sum', 'mean', 'count'],
    fill_value=0
)
pivot_table_enhanced = pd.pivot_table(
    merged_df,
    values='service_charge',
    index='procedure_code',
    columns='region',
    aggfunc=['sum', 'mean', 'count'],
    fill_value=0,
    margins=True,
    margins_name='Total'
)

# Tip: When saving a pivot table to CSV, the index (row labels) is preserved by default.
#      This is usually desired - when reading back, use index_col=0 to restore the index.
# TODO: Save results as 'output/q3_pivot_analysis.csv'
pivot_table_enhanced.to_csv('output/q3_pivot_analysis.csv')

TypeError: 'DataFrame' object is not callable

### Part 3.2: Cross-Tabulations

**TODO: Create cross-tabulations for categorical analysis**

In [46]:
# TODO: Create crosstab of facility type vs region with counts
# TODO: Create crosstab with margins (row and column totals)
# TODO: Create multi-dimensional crosstab showing facility type × encounter type × region combinations
#       (Hint: use pd.crosstab with multiple index/column parameters, or create manually from the data)
facility_x_region = pd.crosstab(
    merged_df['facility_type'],
    merged_df['region'],
    margins=True,
    margins_name='Total'
)
print("========facility_x_region========")
display(facility_x_region)

multi_dim_crosstab = pd.crosstab(
    [merged_df['facility_type'], merged_df['encounter_type']],
    merged_df['region'],
    margins=True,
    margins_name='Total'
)
print("========multi_dim_crosstab========")
display(multi_dim_crosstab)

# TODO: Analyze the cross-tabulation results
# TODO: Save results as 'output/q3_crosstab_analysis.csv'

========facility_x_region========


region,North,South,West,Total
facility_type,,,,
Hospital,0,509,0,509
Specialty Center,0,1026,946,1972
Urgent Care,510,970,1039,2519
Total,510,2505,1985,5000


========multi_dim_crosstab========


region                                 North  South  West  Total
facility_type    encounter_type                                 
Hospital         Emergency                 0    127     0    127
                 Inpatient                 0     95     0     95
                 Lab                       0    100     0    100
                 Office Visit              0     85     0     85
                 Outpatient Procedure      0    102     0    102
Specialty Center Emergency                 0    236   182    418
                 Inpatient                 0    210   191    401
                 Lab                       0    211   180    391
                 Office Visit              0    175   200    375
                 Outpatient Procedure      0    194   193    387
Urgent Care      Emergency               109    190   195    494
                 Inpatient                90    190   201    481
                 Lab                      90    229   228    547
                 Office Visit            112    178   200    490
                 Outpatient Procedure    109    183   215    507
Total                                    510   2505  1985   5000

### Part 3.3: Pivot Table Visualization

**TODO: Create visualizations from pivot tables**

In [ ]:
# TODO: Create heatmap from pivot table
# TODO: Create bar chart from pivot table
# TODO: Customize colors and styling
# TODO: Add appropriate titles and labels
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

sns.heatmap(
    pivot_table['sum'], 
    annot=True, 
    fmt='.0f', 
    cmap='YlGnBu', 
    ax=axes[0],
    cbar_kws={'label': 'Total Charges ($)'}
)
axes[0].set_title('Total Encounter Charges by Procedure Code and Region', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Region', fontsize=12)
axes[0].set_ylabel('Procedure Code', fontsize=12)

# Bar chart - plot each region as a grouped bar
pivot_table['sum'].plot(kind='bar', ax=axes[1], width=0.8, color=['skyblue', 'lightcoral', 'lightgreen', 'plum'])
axes[1].set_title('Total Encounter Charges by Procedure Code and Region', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Procedure Code', fontsize=12)
axes[1].set_ylabel('Total Encounter Charges', fontsize=12)
axes[1].legend(title='Region', loc='upper right')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('output/q3_pivot_visualization.png')
plt.show()

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (2260557947.py, line 8)

## Submission Checklist

Before submitting, verify you've created:

- [ ] `output/q1_groupby_analysis.csv` - Basic groupby analysis
- [ ] `output/q1_aggregation_report.txt` - Aggregation report
- [ ] `output/q2_filter_analysis.csv` - Filter operations analysis
- [ ] `output/q2_hierarchical_analysis.csv` - Hierarchical analysis
- [ ] `output/q2_performance_report.txt` - Performance report
- [ ] `output/q3_pivot_analysis.csv` - Pivot table analysis
- [ ] `output/q3_crosstab_analysis.csv` - Cross-tabulation analysis
- [ ] `output/q3_pivot_visualization.png` - Pivot visualization

## Key Learning Objectives

- Master the split-apply-combine paradigm
- Apply aggregation functions and transformations
- Create pivot tables for multi-dimensional analysis
- Apply advanced groupby techniques